In [1]:
from model import SubsequenceRNN
from datautils import get_data
from torch.utils.data import DataLoader
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda_is_available() else 'cpu')

In [2]:

train, test, y = get_data('126', 400)
train = torch.Tensor(train)
test = torch.Tensor(test)
y = torch.Tensor(y)
print(train.shape, test.shape, y.shape)
# Generators
train_dataloader = DataLoader(train, batch_size=10000, shuffle=True)
test_dataloader = DataLoader((test, y), batch_size=10000, shuffle=True)


[[   0    1    2 ...  397  398  399]
 [   1    2    3 ...  398  399  400]
 [   2    3    4 ...  399  400  401]
 ...
 [7596 7597 7598 ... 7993 7994 7995]
 [7597 7598 7599 ... 7994 7995 7996]
 [7598 7599 7600 ... 7995 7996 7997]]
torch.Size([7599, 400]) torch.Size([55, 400]) torch.Size([55])


In [3]:
model = SubsequenceRNN((400), 5, n_subsequences=10, ranges=(50,250))
model.to(device)

(50, 250)
[173 249  59 218  64 168  71 236  50 228]
400
[(2, 173), (1, 249), (6, 59), (1, 218), (6, 64), (2, 168), (5, 71), (1, 236), (8, 50), (1, 228)]
[GRU(173, 10, num_layers=2, batch_first=True), GRU(249, 10, num_layers=2, batch_first=True), GRU(59, 10, num_layers=2, batch_first=True), GRU(218, 10, num_layers=2, batch_first=True), GRU(64, 10, num_layers=2, batch_first=True), GRU(168, 10, num_layers=2, batch_first=True), GRU(71, 10, num_layers=2, batch_first=True), GRU(236, 10, num_layers=2, batch_first=True), GRU(50, 10, num_layers=2, batch_first=True), GRU(228, 10, num_layers=2, batch_first=True)]
Linear(in_features=330, out_features=5, bias=True)


In [4]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters())

In [ ]:

for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    model.train()
    
    batch_loss_hist = []
    for batch, X in enumerate(train_dataloader):
        X = X.to(device)
        pred = model(X)
        batch_loss = criterion(pred, X)
        
        optimizer.zero_grad() 
        batch_loss.backward()
        optimizer.step()
        batch_loss_hist.append(batch_loss.item())
    print('batch', t,'loss', np.mean(np.array(batch_loss_hist)))
    # Zero gradients, perform a backward pass, and update the weights.


# print(model.rnns)

batch 0 loss 505158.3125
batch 1 loss 486936.40625
batch 2 loss 473580.25
batch 3 loss 464449.125
batch 4 loss 459342.5625
batch 5 loss 457183.90625
batch 6 loss 456483.0
batch 7 loss 456145.53125
batch 8 loss 455450.3125
batch 9 loss 453975.46875
batch 10 loss 451665.40625
batch 11 loss 448770.09375
batch 12 loss 445622.8125
batch 13 loss 442477.4375


In [ ]:
test = test.to(device)
y_pred = model(test)
recon_loss = torch.mean((test - y_pred)**2, axis=1)
print(recon_loss.shape, recon_loss.max(), recon_loss.min())

In [ ]:
pred_anomaly_idx = torch.argmax(recon_loss)

print(torch.argmax(recon_loss).detach().numpy(), torch.argwhere(y == 1))

In [ ]:
_, ax = plt.subplots(2,1,figsize=(20,10))
ax[0].step(np.arange(55), recon_loss.detach().numpy())
ax[1].step(np.arange(55), y.detach().numpy())
plt.show()